In [ ]:

import torch
import torch.nn as nn
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:
def simple_maxwell_model(epsilon, t, G, eta):
    """ 简单的麦克斯韦模型的应力计算 """
    # 计算应变的时间导数
    epsilon_dot = torch.autograd.grad(epsilon, t, grad_outputs=torch.ones_like(epsilon), create_graph=True)[0]
    # 计算应力
    sigma = G * epsilon + eta * epsilon_dot
    return sigma

In [ ]:
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers)-1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
            if i < len(layers)-2:
                self.layers.append(nn.Tanh())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
def net_r(model, gamma_dot, t, T, G, eta):
    """ Residual calculation for simple Maxwell model """
    # 计算应变
    epsilon = model(torch.cat([gamma_dot, t, T], dim=1))

    # 计算应力
    sigma = simple_maxwell_model(epsilon, t, G, eta)

    # 计算残差
    r = sigma - eta * gamma_dot
    return r

In [ ]:
def boundary_loss(model, gamma_dot_b, t_b, T_b, sigma_b, G, eta):
    """ Boundary condition loss """
    epsilon_b = model(torch.cat([gamma_dot_b, t_b, T_b], dim=1))
    sigma_pred = simple_maxwell_model(epsilon_b, t_b, G, eta)
    loss_b = torch.mean((sigma_pred - sigma_b)**2)
    return loss_b

In [ ]:
def loss_func(model, gamma_dot, t, T, gamma_dot_b, t_b, T_b, sigma_b, G, eta):
    """ Joint loss function """
    r = net_r(model, gamma_dot, t, T, G, eta)
    loss_r = torch.mean(r**2)
    loss_b = boundary_loss(model, gamma_dot_b, t_b, T_b, sigma_b, G, eta)
    loss = loss_r + loss_b
    return loss

In [ ]:
# 生成数据
N_r = 1000  # 残差点数量
gamma_dot = torch.linspace(0, 1, N_r).view(-1, 1).requires_grad_(True).to(device)
t = torch.linspace(0, 1, N_r).view(-1, 1).requires_grad_(True).to(device)
T = torch.linspace(20, 100, N_r).view(-1, 1).requires_grad_(True).to(device)  # 温度

# 生成边界条件数据
N_b = 100  # 边界条件点数量
gamma_dot_b = torch.linspace(0, 1, N_b).view(-1, 1).requires_grad_(True).to(device)
t_b = torch.zeros(N_b, 1).requires_grad_(True).to(device)
T_b = torch.linspace(20, 100, N_b).view(-1, 1).requires_grad_(True).to(device)  # 温度
sigma_b = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5] * 20).view(-1, 1).to(device)  # 边界条件应力

# 定义简单的麦克斯韦模型的参数
G = torch.tensor(1.0).to(device)  # 弹性模量
eta = torch.tensor(0.1).to(device)  # 粘度

In [ ]:
# 初始化模型
layers = [3, 20, 20, 20, 1]  # 输入维度为3，输出维度为1，表示应变
model = PINN(layers).to(device)

# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# 训练模型
for epoch in range(1000):
    optimizer.zero_grad()
    loss = loss_func(model, gamma_dot, t, T, gamma_dot_b, t_b, T_b, sigma_b, G, eta)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
# 预测
model.eval()
# 生成均匀变化的模拟数据
gamma_dot_test = torch.linspace(0.1, 10.0, 100, requires_grad=True, device=device).view(-1, 1)
t_test = torch.linspace(0.1, 10.0, 100, requires_grad=True, device=device).view(-1, 1)
T_test = torch.linspace(30.0, 70.0, 100, requires_grad=True, device=device).view(-1, 1)
epsilon_pred = model(torch.cat([gamma_dot_test, t_test, T_test], dim=1))
sigma_pred = simple_maxwell_model(epsilon_pred, t_test, G, eta)

# 调试步骤：打印数据范围
print("t_test range:", t_test.min().item(), t_test.max().item())
print("sigma_pred range:", sigma_pred.min().item(), sigma_pred.max().item())

# 绘制预测结果
plt.figure(figsize=(10, 6))
plt.scatter(t_test.detach().numpy(), sigma_pred.detach().numpy(), label='Predicted Stress')
print(t_test.detach().numpy())
print(sigma_pred.detach().numpy())
plt.xlabel('Time')
plt.ylabel('Stress')
plt.title('Stress vs Time')
plt.legend()
plt.show()